<a href="https://colab.research.google.com/github/viezerb/music-generation/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from music21 import converter, instrument, note, chord
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, BatchNormalization as BatchNorm 
from tensorflow.keras.utils import to_categorical
import glob
import numpy
import pickle
from keras.callbacks import ModelCheckpoint
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [19]:
def get_notes():
    notes = []

    for file in zip(glob.glob("/content/gdrive/MyDrive/music-generation/EMOPIA_1.0/midis/*mid")):
        print("Parsing %s" % file)
        midi = converter.parse(file)

        notes_to_parse = None
        
        try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes
        
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    print(notes)

    return notes


In [5]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100


    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
            
    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

    # normalize input
    network_input = network_input / float(n_vocab)
    network_output = to_categorical(network_output)

    return (network_input, network_output)

In [6]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """

    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model
    

In [7]:
def train(model, network_input, network_output):
    """ train the neural network """

    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=100, batch_size=256, callbacks=callbacks_list)


In [20]:

notes = get_notes()

# get amount of pitch names 
n_vocab = len(set(notes))

network_input, network_output = prepare_sequences(notes, n_vocab)

model = create_network(network_input, n_vocab)

train(model, network_input, network_output)


Parsing /content/gdrive/MyDrive/music-generation/EMOPIA_1.0/midis/Q1_aYe-2Glruu4_3.mid
Parsing /content/gdrive/MyDrive/music-generation/EMOPIA_1.0/midis/Q1_aYe-2Glruu4_1.mid
Parsing /content/gdrive/MyDrive/music-generation/EMOPIA_1.0/midis/Q1_BGf5nCdzPOc_0.mid
Parsing /content/gdrive/MyDrive/music-generation/EMOPIA_1.0/midis/Q1_9v2WSpn4FCw_2.mid
Parsing /content/gdrive/MyDrive/music-generation/EMOPIA_1.0/midis/Q1_9v2WSpn4FCw_6.mid
Parsing /content/gdrive/MyDrive/music-generation/EMOPIA_1.0/midis/Q1_dfNdpy8TUzA_0.mid
Parsing /content/gdrive/MyDrive/music-generation/EMOPIA_1.0/midis/Q1_9v2WSpn4FCw_5.mid
Parsing /content/gdrive/MyDrive/music-generation/EMOPIA_1.0/midis/Q1_Cg2u_Ldjv8g_0.mid
Parsing /content/gdrive/MyDrive/music-generation/EMOPIA_1.0/midis/Q1__BK2o77sTc0_0.mid
Parsing /content/gdrive/MyDrive/music-generation/EMOPIA_1.0/midis/Q1_GbUV3TXUzeQ_1.mid
Parsing /content/gdrive/MyDrive/music-generation/EMOPIA_1.0/midis/Q1_fey-8bOR95E_0.mid
Parsing /content/gdrive/MyDrive/music-gener

KeyboardInterrupt: ignored